## TRADE DATA

In [26]:
import pandas as pd
import datetime

def process_csv(filepath, product_index):
    """
    This function takes the file path and product index as inputs and returns the filtered trade data dataframe
    for a particular product.
    
    Parameters:
    filepath (str) : file path for the trade data file
    product_index (int) : index of the product in the list of all products
    
    Returns:
    df_product_filtered (pandas dataframe) : dataframe for the filtered product
    """
    
    # Read the CSV file and convert the "TIME_PERIOD" column to datetime
    trade_data = pd.read_csv(filepath)
    trade_data["TIME_PERIOD"] = pd.to_datetime(trade_data["TIME_PERIOD"], format = '%Y-%m').dt.date
    
    # Rename columns and remove duplicates
    trade_data = trade_data.rename(columns={"partner": "PARTNER_codes", "declarant": "DECLARANT_codes"})
    trade_data = trade_data.drop_duplicates()
    
    # Get the list of all products
    products = trade_data["product"].drop_duplicates().to_list()
    
    # Filter the data for a particular product
    product_filter = products[product_index]
    
    # Read the partner countries data
    partner_countries = pd.read_csv("trade_data/partners.csv", sep = ";")
    
    # Merge the trade data and partner country data on the "PARTNER_codes" column
    trade_data = pd.merge(trade_data, partner_countries, on = ['PARTNER_codes'], how = 'inner')
    
    # Pivot the trade data to create a multi-index dataframe
    trade_data_pivot = trade_data.pivot(index=['PARTNER_Labels', 'TIME_PERIOD', 'product'], 
                                  columns='indicators', 
                                  values=['OBS_VALUE'])
    
    # Flatten the column names
    trade_data_pivot.columns = trade_data_pivot.columns.map('_'.join)
    
    # Reset the index and remove missing values
    trade_data_pivot = trade_data_pivot.reset_index()
    trade_data_pivot = trade_data_pivot.dropna()
    
    # Filter the data for the selected product
    df = trade_data_pivot
    df_product_filtered = df[df['product'] == product_filter]

    
    return df_product_filtered

#Declare DataFrames 
df_maize_corn_0 = process_csv("trade_data/maize_corn.csv", 0)
df_maize_corn_0['price'] = df_maize_corn_0['OBS_VALUE_VALUE_1000EURO'] / df_maize_corn_0['OBS_VALUE_QUANTITY_TON']

# Keep '10059000' the same but name it 'maize'
df_maize = df_maize_corn_0
df_maize['product'] = 'maize'


In [74]:
df_maize

,PARTNER_Labels,TIME_PERIOD,product,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price
0,Afghanistan,2021-03-01,maize,0.0,0.04,inf
1,Antigua and Barbuda,2020-01-01,maize,54.0,21.20,0.392593
2,Antigua and Barbuda,2020-04-01,maize,27.0,12.14,0.449630
3,Argentina,2005-01-01,maize,8100.0,937.50,0.115741
4,Argentina,2005-02-01,maize,3172.0,392.55,0.123755
...,...,...,...,...,...,...
5534,Zimbabwe,2007-01-01,maize,0.1,0.13,1.300000
5535,Zimbabwe,2010-12-01,maize,0.3,1.11,3.700000
5536,Zimbabwe,2011-01-01,maize,0.1,0.38,3.800000
5537,Zimbabwe,2011-03-01,maize,0.2,0.38,1.900000


## ANALYSIS CORN IMPORT TO NL (HISTORICAL)

In [87]:
df_maize_corn_0['TIME_PERIOD'] = pd.to_datetime(df_maize_corn_0['TIME_PERIOD'])
mask = (df_maize_corn_0['TIME_PERIOD'] >= '2005-01-01') & (df_maize_corn_0['TIME_PERIOD'] <= '2022-10-01')
df_filtered = df_maize_corn_0.loc[mask]
total = df_filtered['OBS_VALUE_QUANTITY_TON'].sum()

total

77296299.4

In [88]:
df_maize_corn_0['TIME_PERIOD'] = pd.to_datetime(df_maize_corn_0['TIME_PERIOD'])
mask = (df_maize_corn_0['TIME_PERIOD'] >= '2005-01-01') & (df_maize_corn_0['TIME_PERIOD'] <= '2022-10-01')
df_filtered = df_maize_corn_0.loc[mask]

grouped = df_filtered.groupby('PARTNER_Labels')['OBS_VALUE_QUANTITY_TON'].sum().reset_index()
grouped = grouped.sort_values(by='OBS_VALUE_QUANTITY_TON', ascending=False)
grouped['cumulative_sum'] = grouped['OBS_VALUE_QUANTITY_TON'].cumsum()
grouped['cumulative_sum_pct'] = grouped['cumulative_sum'] / grouped['OBS_VALUE_QUANTITY_TON'].sum()

grouped.head(50)        
        
#print("The top", top_x, "PARTNER_Labels cover 80% of the market.")

,PARTNER_Labels,OBS_VALUE_QUANTITY_TON,cumulative_sum,cumulative_sum_pct
101,Ukraine,22670363.9,22670363.9,0.293292
32,France,20818695.8,43489059.7,0.562628
9,Brazil,7514296.3,51003356.0,0.659842
35,Germany,7132645.5,58136001.5,0.752119
40,Hungary,3995159.3,62131160.8,0.803805
79,Romania,3816059.5,65947220.3,0.853174
6,Belgium,3008019.1,68955239.4,0.892090
80,Russia,1464272.2,70419511.6,0.911033
2,Argentina,1357260.3,71776771.9,0.928593
104,United States,1045888.5,72822660.4,0.942124


## WEATHER DATA

In [33]:
import pandas as pd
import datetime

#Clean import oil_price_index_2015_oecd

# Read the data from the weather file
df_global_temperature = pd.read_csv('weather_data/global_temperature_data.csv')

# Rename the 'YearMonth' column to 'TIME_PERIOD'
df_global_temperature.rename(columns={'yearmonth': 'TIME_PERIOD', 'partner_labels': 'PARTNER_Labels'}, inplace=True)

# Convert the TIME_PERIOD column to datetime format
df_global_temperature['TIME_PERIOD'] = pd.to_datetime(df_global_temperature['TIME_PERIOD'], format='%Y%m')

# Convert the TIME_PERIOD column to the desired format
df_global_temperature['TIME_PERIOD'] = df_global_temperature['TIME_PERIOD'].dt.strftime('%Y-%m-%d')

# Drop columns
df_global_temperature.drop(columns=['partner_code', 'weather_countrycode', 'weather_countryname', 'iso_countrycode'], inplace=True)

df_global_temperature['TIME_PERIOD'] = pd.to_datetime(df_global_temperature['TIME_PERIOD'])
df_global_temperature

,PARTNER_Labels,TIME_PERIOD,AVG_TAVG,MIN_TMIN,MAX_TMAX
0,United Arab Emirates,2005-01-01,186.096154,110.000000,280.000000
1,United Arab Emirates,2005-02-01,204.750000,106.000000,306.500000
2,United Arab Emirates,2005-03-01,238.226667,124.500000,363.666667
3,United Arab Emirates,2005-04-01,280.984568,155.000000,423.333333
4,United Arab Emirates,2005-05-01,321.296296,192.000000,435.500000
...,...,...,...,...,...
21085,Viet Nam,2022-08-01,287.060676,244.363636,348.666667
21086,Viet Nam,2022-09-01,279.751352,233.600000,350.000000
21087,Viet Nam,2022-10-01,246.000000,238.000000,260.000000
21088,Viet Nam,2022-11-01,272.000000,252.000000,304.000000


## OIL DATA

In [34]:
import pandas as pd
import datetime

#Clean import oil_price_index_2015_oecd

# Read the data from the 'oil_price_index_2015_oecd.xlsx' file
df_oil_index = pd.read_excel('oil_prices/oil_price_index_2015_oecd.xlsx', skiprows=[0,1,2,3,4, 9012, 9013, 9014, 9015])
# Forward fill the 'Country' column
df_oil_index['Country'].fillna(method='ffill', inplace=True)
# Rename columns
df_oil_index.rename(columns={'Unnamed: 3': 'CPI_ENERGY'}, inplace=True)
df_oil_index.rename(columns={'Unnamed: 4': 'CPI_ENERGY_HARMONISED'}, inplace=True)
# Delete the 'Unnamed: 2' column
df_oil_index.drop(columns=['Unnamed: 2'], inplace=True)
# Convert the 'Time' column to datetime format
df_oil_index['Time'] = pd.to_datetime(df_oil_index['Time'], format='%b-%Y')
df_oil_index.rename(columns={'Time': 'TIME_INDEX'}, inplace=True)

#Clean import CMO-Historical-Data-Monthly.xlsx

df_commodities_price = pd.read_excel('oil_prices/CMO-Historical-Data-Monthly.xlsx', sheet_name = 'Monthly Prices', skiprows=[0,1,2,3,4,5])
# Rename columns
df_commodities_price.rename(columns={'Unnamed: 0': 'TIME_PERIOD'}, inplace=True)
df_commodities_price.rename(columns={'CRUDE_PETRO': 'OIL_PRICE $/bbl'}, inplace=True)
df_commodities_price['TIME_PERIOD'] = pd.to_datetime(df_commodities_price['TIME_PERIOD'], format='%YM%m')

df_oil_price_full = pd.DataFrame(df_commodities_price, columns=['TIME_PERIOD', 'OIL_PRICE $/bbl'])

df_oil_price = df_oil_price_full.loc[(df_oil_price_full['TIME_PERIOD'] >= '2005-01-01') & (df_oil_price_full['TIME_PERIOD'] <= '2022-10-01')]
df_oil_price

,TIME_PERIOD,OIL_PRICE $/bbl
540,2005-01-01,42.972278
541,2005-02-01,44.818211
542,2005-03-01,50.942879
543,2005-04-01,50.640476
544,2005-05-01,47.826573
...,...,...
749,2022-06-01,116.800000
750,2022-07-01,105.083333
751,2022-08-01,95.973333
752,2022-09-01,88.220000


## FUTURES PRICES (CORN)

In [65]:
import pandas as pd
import datetime

#Clean import oil_price_index_2015_oecd

# Read the data from the weather file
df_futures_corn = pd.read_csv('futures_data/US Corn Futures Historical Data.csv')

# Rename the 'date' column to 'TIME_PERIOD'
df_futures_corn.rename(columns={'Date': 'TIME_PERIOD', 'Price': 'Corn Price Futures'}, inplace=True)

df_futures_corn['TIME_PERIOD'] = pd.to_datetime(df_futures_corn['TIME_PERIOD'], format='%d/%m/%Y')
df_futures_corn['TIME_PERIOD'] = df_futures_corn['TIME_PERIOD'].dt.strftime('%Y/%d/%m')

df_futures_corn.drop(columns=['Open','High','Low','Vol.','Change %'], inplace=True)

df_futures_corn['TIME_PERIOD'] = pd.to_datetime(df_futures_corn['TIME_PERIOD'])

df_futures_corn.sort_values(by='TIME_PERIOD', ascending=True, inplace=True)
df_futures_corn

,TIME_PERIOD,Corn Price Futures
212,2005-02-01,222.75
211,2005-03-01,221.00
210,2005-04-01,213.50
209,2005-05-01,230.75
208,2005-06-01,222.25
...,...,...
4,2022-06-01,743.75
3,2022-07-01,616.25
2,2022-08-01,673.75
1,2022-09-01,677.50


## Merged trade data, oil price

In [66]:
df_maize['TIME_PERIOD'] = pd.to_datetime(df_maize['TIME_PERIOD'])
df_merged_trade_oil = df_maize.merge(df_oil_price, on='TIME_PERIOD', how='left')
df_merged_trade_oil

,PARTNER_Labels,TIME_PERIOD,product,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,OIL_PRICE $/bbl
0,Afghanistan,2021-03-01,maize,0.0,0.04,inf,63.830000
1,Antigua and Barbuda,2020-01-01,maize,54.0,21.20,0.392593,61.626667
2,Antigua and Barbuda,2020-04-01,maize,27.0,12.14,0.449630,21.043333
3,Argentina,2005-01-01,maize,8100.0,937.50,0.115741,42.972278
4,Argentina,2005-02-01,maize,3172.0,392.55,0.123755,44.818211
...,...,...,...,...,...,...,...
5534,Zimbabwe,2007-01-01,maize,0.1,0.13,1.300000,53.516970
5535,Zimbabwe,2010-12-01,maize,0.3,1.11,3.700000,90.005962
5536,Zimbabwe,2011-01-01,maize,0.1,0.38,3.800000,92.690595
5537,Zimbabwe,2011-03-01,maize,0.2,0.38,1.900000,108.645217


## Merged trade data, oil price, weather data

In [67]:
df_merged_trade_oil_weather = df_merged_trade_oil.merge(df_global_temperature, on=['TIME_PERIOD', 'PARTNER_Labels'], how='left')
df_merged_trade_oil_weather

,PARTNER_Labels,TIME_PERIOD,product,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,OIL_PRICE $/bbl,AVG_TAVG,MIN_TMIN,MAX_TMAX
0,Afghanistan,2021-03-01,maize,0.0,0.04,inf,63.830000,NaN,NaN,NaN
1,Antigua and Barbuda,2020-01-01,maize,54.0,21.20,0.392593,61.626667,NaN,NaN,NaN
2,Antigua and Barbuda,2020-04-01,maize,27.0,12.14,0.449630,21.043333,NaN,NaN,NaN
3,Argentina,2005-01-01,maize,8100.0,937.50,0.115741,42.972278,NaN,NaN,NaN
4,Argentina,2005-02-01,maize,3172.0,392.55,0.123755,44.818211,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
5534,Zimbabwe,2007-01-01,maize,0.1,0.13,1.300000,53.516970,NaN,NaN,NaN
5535,Zimbabwe,2010-12-01,maize,0.3,1.11,3.700000,90.005962,NaN,NaN,NaN
5536,Zimbabwe,2011-01-01,maize,0.1,0.38,3.800000,92.690595,NaN,NaN,NaN
5537,Zimbabwe,2011-03-01,maize,0.2,0.38,1.900000,108.645217,NaN,NaN,NaN


## Merged trade data, oil price, weather, future prices data

In [68]:
df_merged = df_merged_trade_oil_weather.merge(df_futures_corn, on='TIME_PERIOD', how='left')
df_merged

,PARTNER_Labels,TIME_PERIOD,product,OBS_VALUE_QUANTITY_TON,OBS_VALUE_VALUE_1000EURO,price,OIL_PRICE $/bbl,AVG_TAVG,MIN_TMIN,MAX_TMAX,Corn Price Futures
0,Afghanistan,2021-03-01,maize,0.0,0.04,inf,63.830000,NaN,NaN,NaN,564.25
1,Antigua and Barbuda,2020-01-01,maize,54.0,21.20,0.392593,61.626667,NaN,NaN,NaN,381.25
2,Antigua and Barbuda,2020-04-01,maize,27.0,12.14,0.449630,21.043333,NaN,NaN,NaN,311.50
3,Argentina,2005-01-01,maize,8100.0,937.50,0.115741,42.972278,NaN,NaN,NaN,NaN
4,Argentina,2005-02-01,maize,3172.0,392.55,0.123755,44.818211,NaN,NaN,NaN,222.75
...,...,...,...,...,...,...,...,...,...,...,...
5534,Zimbabwe,2007-01-01,maize,0.1,0.13,1.300000,53.516970,NaN,NaN,NaN,415.75
5535,Zimbabwe,2010-12-01,maize,0.3,1.11,3.700000,90.005962,NaN,NaN,NaN,636.50
5536,Zimbabwe,2011-01-01,maize,0.1,0.38,3.800000,92.690595,NaN,NaN,NaN,670.00
5537,Zimbabwe,2011-03-01,maize,0.2,0.38,1.900000,108.645217,NaN,NaN,NaN,701.00


## Merged trade, oil price, weather, future prices, macroeconomic 

## End result for now

In [69]:
###FOR NOW
df_maize_corn_trade_weather_futures = df_merged.pivot_table(index='TIME_PERIOD', columns=['PARTNER_Labels'], values=['OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO', 'price', 'OIL_PRICE $/bbl', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX', 'Corn Price Futures'])
df_maize_corn_trade_weather_futures

AVG_TAVG                                                 \
PARTNER_Labels   Australia     Austria     Belgium Bosnia and Herzegovina   
TIME_PERIOD                                                                 
2005-01-01             NaN         NaN   30.700000                    NaN   
2005-02-01             NaN         NaN   20.000000                    NaN   
2005-03-01             NaN   -1.994624   55.800000                    NaN   
2005-04-01             NaN   60.233333  111.000000                    NaN   
2005-05-01             NaN  108.758065  129.250000                    NaN   
...                    ...         ...         ...                    ...   
2022-06-01      137.954664  182.161522  180.681818                    NaN   
2022-07-01             NaN  177.245503  190.000000                    NaN   
2022-08-01             NaN  175.258849  213.200000                    NaN   
2022-09-01             NaN  103.779082  166.000000                    NaN   
2022-10-01             NaN   98.433871  147.333333                    NaN   

                                                                               \
PARTNER_Labels      Brazil    Bulgaria Cameroon      Canada Chile       China   
TIME_PERIOD                                                                     
2005-01-01             NaN         NaN      NaN         NaN   NaN         NaN   
2005-02-01      266.743474         NaN      NaN         NaN   NaN         NaN   
2005-03-01             NaN         NaN      NaN         NaN   NaN         NaN   
2005-04-01             NaN         NaN      NaN         NaN   NaN         NaN   
2005-05-01             NaN         NaN      NaN         NaN   NaN         NaN   
...                    ...         ...      ...         ...   ...         ...   
2022-06-01      215.582738  219.847222      NaN  141.197971   NaN  222.902520   
2022-07-01      227.424074  245.580645      NaN  176.170636   NaN         NaN   
2022-08-01      222.129167  243.446237      NaN  179.322868   NaN  234.647872   
2022-09-01      221.270833  180.200000      NaN  131.132246   NaN         NaN   
2022-10-01      242.066667  138.333333      NaN         NaN   NaN         NaN   

                ...     price                                        \
PARTNER_Labels  ...   Türkiye Uganda   Ukraine United Arab Emirates   
TIME_PERIOD     ...                                                   
2005-01-01      ...  0.710000    NaN       NaN                  NaN   
2005-02-01      ...       NaN    NaN       NaN                  NaN   
2005-03-01      ...       NaN    NaN       NaN                  NaN   
2005-04-01      ...       NaN    NaN       inf                  NaN   
2005-05-01      ...       NaN    NaN       NaN                  NaN   
...             ...       ...    ...       ...                  ...   
2022-06-01      ...  0.424940    NaN  0.383358                  NaN   
2022-07-01      ...  0.350000    NaN  0.359280                  NaN   
2022-08-01      ...  0.300000    NaN  0.357720                  NaN   
2022-09-01      ...  0.428571    inf  0.313625                  NaN   
2022-10-01      ...  0.442500    NaN  0.313023                  NaN   

                                                                              
PARTNER_Labels United Kingdom United States Uruguay Viet Nam Zambia Zimbabwe  
TIME_PERIOD                                                                   
2005-01-01           0.314011      0.334963     NaN      NaN    NaN    3.080  
2005-02-01           0.380808      0.270379     NaN      NaN    NaN    2.550  
2005-03-01           0.329100      0.274034     NaN      NaN    NaN    1.900  
2005-04-01           0.292759      0.365034     NaN      NaN    NaN    0.775  
2005-05-01           0.336029      0.298552     NaN      NaN    NaN      inf  
...                       ...           ...     ...      ...    ...      ...  
2022-06-01           0.460000      0.424965     NaN      NaN    NaN      NaN  
2022-07-01         

## Save excel file

In [70]:
# Set the path and name of the Excel file
file_path = "df_maize_trade_oil_weather_futures.xlsx"

# Write the df_pivot dataframe to the specified Excel file
df_maize_corn_trade_weather_futures.to_excel(file_path, index=True)

## Simplicity/overview: filter top 5 countries

In [89]:
# Filter the data to include only the desired countries
df_filtered = df_merged[df_merged['PARTNER_Labels'].isin(['Ukraine', 'France', 'Brazil', 'Germany', 'Hungary'])]

# Create the pivot table
df_filtered_maize_corn_trade_weather_futures = df_filtered.pivot_table(index='TIME_PERIOD', columns=['PARTNER_Labels'], values=['OBS_VALUE_QUANTITY_TON', 'OBS_VALUE_VALUE_1000EURO', 'price', 'OIL_PRICE $/bbl', 'AVG_TAVG', 'MIN_TMIN', 'MAX_TMAX', 'Corn Price Futures'])
df_filtered_maize_corn_trade_weather_futures

AVG_TAVG                                                  \
PARTNER_Labels      Brazil      France     Germany     Hungary     Ukraine   
TIME_PERIOD                                                                  
2005-01-01             NaN   50.081198   28.651592   -1.306452         NaN   
2005-02-01      266.743474   30.689158   -1.962406  -25.371429         NaN   
2005-03-01             NaN   76.202630   45.730051   38.383871         NaN   
2005-04-01             NaN  108.081105  101.859649  112.460000   90.622214   
2005-05-01             NaN  147.068032  134.667233  162.993548         NaN   
...                    ...         ...         ...         ...         ...   
2022-06-01      215.582738  197.346344  188.737990  225.287222  209.451222   
2022-07-01      227.424074  220.948883  198.723435  240.073214  212.861598   
2022-08-01      222.129167  221.881986  208.800878  237.433908  224.523306   
2022-09-01      221.270833  170.650565  139.969363  182.377451  136.176826   
2022-10-01      242.066667  159.565015  130.116343  145.500000  113.858268   

               Corn Price Futures                                  ...  \
PARTNER_Labels             Brazil  France Germany Hungary Ukraine  ...   
TIME_PERIOD                                                        ...   
2005-01-01                    NaN     NaN     NaN     NaN     NaN  ...   
2005-02-01                 222.75  222.75  222.75  222.75     NaN  ...   
2005-03-01                    NaN  221.00  221.00  221.00     NaN  ...   
2005-04-01                    NaN  213.50  213.50  213.50  213.50  ...   
2005-05-01                    NaN  230.75  230.75  230.75     NaN  ...   
...                           ...     ...     ...     ...     ...  ...   
2022-06-01                 743.75  743.75  743.75  743.75  743.75  ...   
2022-07-01                 616.25  616.25  616.25  616.25  616.25  ...   
2022-08-01                 673.75  673.75  673.75  673.75  673.75  ...   
2022-09-01                 677.50  677.50  677.50  677.50  677.50  ...   
2022-10-01                 691.50  691.50  691.50  691.50  691.50  ...   

               OIL_PRICE $/bbl                                      \
PARTNER_Labels          Brazil      France     Germany     Hungary   
TIME_PERIOD                                                          
2005-01-01                 NaN   42.972278   42.972278   42.972278   
2005-02-01           44.818211   44.818211   44.818211   44.818211   
2005-03-01                 NaN   50.942879   50.942879   50.942879   
2005-04-01                 NaN   50.640476   50.640476   50.640476   
2005-05-01                 NaN   47.826573   47.826573   47.826573   
...                        ...         ...         ...         ...   
2022-06-01          116.800000  116.800000  116.800000  116.800000   
2022-07-01          105.083333  105.083333  105.083333  105.083333   
2022-08-01           95.973333   95.973333   95.973333   95.973333   
2022-09-01           88.220000   88.220000   88.220000   88.220000   
2022-10-01           90.326667   90.326667   90.326667   90.326667   

                               price                                          
PARTNER_Labels     Ukraine    Brazil    France   Germany   Hungary   Ukraine  
TIME_PERIOD                                                                   
2005-01-01             NaN       NaN  0.122632  0.120429  0.109604       NaN  
2005-02-01             NaN  0.164250  0.124028  0.109035  0.105578       NaN  
2005-03-01             NaN       NaN  0.123698  0.118774  0.102434       NaN  
2005-04-01       50.640476       NaN  0.117439  0.109957  0.104302       inf  
2005-05-01             NaN       NaN  0.123325  0.116828  0.115436       NaN  
...                    ...       ...       ...       ...       ...       ...  
2022-06-01      116.800000  0.400000  0.277599  0.549137  0.403821  0.383358  
2022-07-01      105.083333  0.351755  0.361186  0.383719  0.366311  0.359280  
2022-08-01       95.973333  0.211752  

In [72]:
df_filtered_maize_corn_trade_weather_futures.to_excel("df_filtered_maize_trade_oil_weather_futures.xlsx", index=True)